In [6]:
import numpy as np
import pandas as pd
from osgeo import gdal
from osgeo import ogr
import os

##tif转nc

In [2]:
inputfile = 'D:\LAI_fitting_param\CI_2007.tif'
outputfile = 'D:\LAI_fitting_param\CI_2007.nc'
#Do not change this line, the following command will convert the geoTIFF to a netCDF
ds = gdal.Translate(outputfile, inputfile, format='NetCDF')

##栅格矢量化

In [9]:
def PolygonizeTheRaster():
    inputfile = r'../dataGdals/landcover_2007_gdals.tif'
    ds = gdal.Open(inputfile, gdal.GA_ReadOnly)
    srcband=ds.GetRasterBand(1)
    maskband=srcband.GetMaskBand()
    dst_filename='../dataGdals/landcover_2007_gdals.shp'
    drv = ogr.GetDriverByName('ESRI Shapefile')
    dst_ds = drv.CreateDataSource(dst_filename)
    srs = None
    dst_layername = 'out'
    dst_layer = dst_ds.CreateLayer(dst_layername, srs=srs)
    dst_fieldname = 'DN'
    fd = ogr.FieldDefn(dst_fieldname, ogr.OFTInteger)
    dst_layer.CreateField(fd)
    dst_field = 0
    # prog_func =test()
    options=[]
    # 参数  输入栅格图像波段\掩码图像波段、矢量化后的矢量图层、需要将DN值写入矢量字段的索引、算法选项、进度条回调函数、进度条参数
    gdal.Polygonize(srcband, maskband, dst_layer,dst_field, options,callback=0)
PolygonizeTheRaster()

##矢量栅格化

In [ ]:
def RasterzeTheVectorToRasterr():
    print("lets gogogogo")
    # 参数说明： 输出的栅格数据，注意该数据必须以update模式打开、指定要更新的波段个数(更新123波段)、指定的图层、几何图形坐标转换图像行列号函数、几何图形坐标转换图像行列号参数、以及图层中属性字段属性值
    inputfilePath='G:/studyprojects/gdal/GdalStudy/Files/plogon/test.shp'
    outputfile='G:/studyprojects/gdal/GdalStudy/Files/images/vector.tif'
    templetefile='G:/studyprojects/gdal/GdalStudy/Files/images/2016CHA.tif'
    data = gdal.Open(templetefile, gdalconst.GA_ReadOnly)
    x_res = data.RasterXSize
    y_res = data.RasterYSize
    vector = ogr.Open(inputfilePath)
    layer = vector.GetLayer()
    targetDataSet = gdal.GetDriverByName('GTiff').Create(outputfile, x_res, y_res, 3, gdal.GDT_Byte)
    #targetDataSet=gdal.GetDriverByName('GTiff').CreateCopy(templetefile,data)
    targetDataSet.SetGeoTransform(data.GetGeoTransform())
    targetDataSet.SetProjection(data.GetProjection())
    band = targetDataSet.GetRasterBand(1)
    NoData_value = -999
    band.SetNoDataValue(NoData_value)
    band.FlushCache()
    gdal.RasterizeLayer(targetDataSet, [1,2,3], layer, options=["ATTRIBUTE=DN"])
